# Fine Tunning using unsloth

In [18]:
import torch
print("CUDA Available: ", torch.cuda.is_available())
print("CUDA Device Name: ", torch.cuda.get_device_name(0))
torch.cuda.empty_cache()

# Verificar se CUDA está disponível para acelerar o processamento
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

CUDA Available:  True
CUDA Device Name:  NVIDIA GeForce RTX 3050 Ti Laptop GPU
Usando dispositivo: cuda


# Define Model

In [3]:
from unsloth import FastLanguageModel
import torch

2024-10-29 14:08:12.485380: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-29 14:08:12.497384: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-29 14:08:12.513186: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-29 14:08:12.517931: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 14:08:12.529997: I tensorflow/core/platform/cpu_feature_guar

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [4]:
initial_model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name="unsloth/Llama-3.2-3B-bnb-4bit",
    model_name = "unsloth/Llama-3.2-3B-Instruct", # or choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # device_map="auto"
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

model = initial_model


==((====))==  Unsloth 2024.10.6: Fast Llama patching. Transformers = 4.46.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU. Max memory: 3.712 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.6. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [5]:
initial_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), ep

In [6]:
# # Mover o modelo para CPU antes de deletar
# model.to("cpu")
# # del model

In [7]:
# import torch
# import gc

# # Remover referência ao modelo
# del model

# # Coleta de lixo para liberar objetos órfãos
# gc.collect()

# # Liberar memória na GPU
# torch.cuda.empty_cache()


# Dataset TeleQnA

## Release 17 Questions

In [8]:
import json

# Path to the TeleQnA processed question in JSON file
rel17_questions_path = r"../Files/rel17_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_questions_path, "r", encoding="utf-8") as file:
    rel17_questions = json.load(file)
print(len(rel17_questions))

733


In [9]:
# Path to the TeleQnA processed question in JSON file
rel17_100_questions_path = r"../Files/rel17_100_questions.json"

# Load the TeleQnA data just release 17
with open(rel17_100_questions_path, "r", encoding="utf-8") as file:
    rel17_100_questions = json.load(file)
print(len(rel17_100_questions))

100


In [10]:
rel17_other_questions = [q for q in rel17_questions if q not in rel17_100_questions]
print(len(rel17_other_questions))

rel17_other_questions_length = 500

rel17_other_questions = rel17_other_questions[:rel17_other_questions_length]


633


In [11]:
rel17_other_questions[0]

{'question': 'Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]',
 'option 1': 'PBCH',
 'option 2': 'PCFICH',
 'option 3': 'PDSCH',
 'option 4': 'PHICH',
 'answer': 'option 2: PCFICH',
 'explanation': 'The physical control format indicator channel (PCFICH) informs the UE and the RN about the number of OFDM symbols used for the PDCCHs.',
 'category': 'Standards specifications'}

## Questions without rel 17 and 18

In [12]:
# Path to the TeleQnA processed question in JSON file
questions_no_rel_17_18_path = r"../Files/questions_no_rel_17_18.json"

# Load the TeleQnA data just release 17
with open(questions_no_rel_17_18_path, "r", encoding="utf-8") as file:
    questions_no_rel_17_18 = json.load(file)
print(len(questions_no_rel_17_18))

8487


In [13]:
questions_no_rel_17_18_length = 1500
questions_no_rel_17_18 = questions_no_rel_17_18[:questions_no_rel_17_18_length]
print(len(questions_no_rel_17_18))

1500


In [14]:
questions_no_rel_17_18[0]

{'question': 'Which non-orthogonal multiple access scheme utilizes the low-complexity message passing algorithm at the receiver for user data detection?',
 'option 1': 'NOMA',
 'option 2': 'PDMA',
 'option 3': 'MUSA',
 'option 4': 'MUST',
 'option 5': 'SCMA',
 'answer': 'option 5: SCMA',
 'explanation': 'The SCMA scheme utilizes the low-complexity message passing algorithm at the receiver for user data detection.',
 'category': 'Research publications'}

## Train Data

In [15]:
# train_questions = rel17_other_questions
train_questions = rel17_other_questions + questions_no_rel_17_18
print(len(train_questions))

2000


In [16]:
train_questions[0]

{'question': 'Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]',
 'option 1': 'PBCH',
 'option 2': 'PCFICH',
 'option 3': 'PDSCH',
 'option 4': 'PHICH',
 'answer': 'option 2: PCFICH',
 'explanation': 'The physical control format indicator channel (PCFICH) informs the UE and the RN about the number of OFDM symbols used for the PDCCHs.',
 'category': 'Standards specifications'}

In [17]:
from datasets import Dataset

# Structure to store pairs of questions and explanations
data = []

half_questions = len(train_questions)//2

# Fill the dataset with (question, explanation) pairs
for item in train_questions[:half_questions]:
    
    human_value = (
        f"{item['question']}"
    )
    
    # Combine the answer and explanation
    gpt_value = (
        f"{item['explanation']}"
    )
    
    # Create a dictionary for each input pair
    pair = [
        {'from': 'human', 'value': human_value},  # For the question
        {'from': 'gpt', 'value': gpt_value}  # For the explanation
    ]
    
    data.append(pair)  # Add the pair to the dataset

data_no_options = data
print(data_no_options[0])

[{'from': 'human', 'value': 'Which physical channel informs the UE and the RN about the number of OFDM symbols used for the PDCCHs? [3GPP Release 17]'}, {'from': 'gpt', 'value': 'The physical control format indicator channel (PCFICH) informs the UE and the RN about the number of OFDM symbols used for the PDCCHs.'}]


In [18]:
from datasets import Dataset

# Structure to store pairs of questions, options, answers, and explanations
data = []

# Fill the dataset with (question + options, answer + explanation) pairs
for item in train_questions[half_questions:]:
    
    # Extract options
    options = [f"{key}: {value}" for key, value in item.items() if 'option' in key]
    # Combine the question and options
    human_value = (
        f"Question: {item['question']}\n"
        f"Options:\n" + "\n".join(options) + "\n"
    )
    
    # Combine the answer and explanation
    gpt_value = (
        f"Answer: {item['answer']}\n"
        f"Explanation: {item['explanation']}"
    )

    # Create a dictionary for each input pair
    pair = [
        {'from': 'human', 'value': human_value},  # Question with options
        {'from': 'gpt', 'value': gpt_value}       # Answer with explanation
    ]
    
    data.append(pair)  # Add the pair to the dataset

# Create the dataset using Hugging Face
data_options = data
print(data_options[0])

[{'from': 'human', 'value': 'Question: What is the key challenge for SDON (Software Defined Optical Network) virtualization?\nOptions:\noption 1: High costs of optical infrastructures\noption 2: Creating multiple VONs over the optical network infrastructure\noption 3: Amortizing costs over relatively few users\noption 4: Accounting for the specific optical transmission and signal propagation characteristics\n'}, {'from': 'gpt', 'value': 'Answer: option 4: Accounting for the specific optical transmission and signal propagation characteristics\nExplanation: The key challenge for SDON virtualization is accounting for the specific optical transmission and signal propagation characteristics.'}]


In [19]:
import random

data_total = data_no_options + data_options
# Shuffle the combined data
random.shuffle(data_total)
print(len(data_total))

2000


In [20]:
# Create the dataset using Hugging Face
# Convert the list of pairs into the appropriate format
formatted_data = {'conversations': data_total}

# Transform the data into a Dataset
dataset = Dataset.from_dict(formatted_data)

# Display information about the created dataset
print(dataset)

# Display the first entry of the dataset
print(dataset[0])

Dataset({
    features: ['conversations'],
    num_rows: 2000
})
{'conversations': [{'from': 'human', 'value': 'What do the TYPE(TAG) and VLAN TPID fields in an IEEE802.1Q tagged frame specify? [IEEE 802.3]'}, {'from': 'gpt', 'value': 'The TYPE(TAG) and VLAN TPID fields in an IEEE802.1Q tagged frame specify the VLAN identification.'}]}


In [21]:
# dataset.save_to_disk('../Files/train_questions_dataset_2000_questions')

In [6]:
from datasets import load_from_disk

dataset_path = '../Files/train_questions_dataset_2000_questions'

dataset = load_from_disk(dataset_path)

print(len(dataset))

2000


In [7]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'What is the purpose of sequence and cyclic shift hopping in PUCCH formats 0, 1, 3, and 4? [3GPP Release 17]'},
  {'from': 'gpt',
   'value': 'Sequence and cyclic shift hopping are used in PUCCH formats 0, 1, 3, and 4 to reduce the interference between PUCCH transmissions. These hopping mechanisms ensure that different UEs use different sequences and cyclic shifts, increasing the orthogonality of the transmitted signals and reducing inter-UE interference.'}]}

In [8]:
dataset[5]

{'conversations': [{'from': 'human',
   'value': 'Question: Which decoding algorithm for low-density parity-check codes involves a complex non-linear function in the check-node processing?\nOptions:\noption 1: Normalized min-sum\noption 2: Off-set min-sum\noption 3: Belief propagation\noption 4: Successive cancellation\noption 5: Sequential decoding\n'},
  {'from': 'gpt',
   'value': 'Answer: option 3: Belief propagation\nExplanation: The belief propagation decoding algorithm of low-density parity-check codes involves a complex non-linear function in the check-node processing.'}]}

# Build TeleQnA Dataset for training

In [9]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

def formatting_prompts_func(examples):
    convos = examples["conversations"]
    texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
    return { "text" : texts, }
pass

In [10]:
from unsloth.chat_templates import standardize_sharegpt
dataset = standardize_sharegpt(dataset)
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [11]:
dataset

Dataset({
    features: ['conversations', 'text'],
    num_rows: 2000
})

In [12]:
dataset[5]['text']

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Which decoding algorithm for low-density parity-check codes involves a complex non-linear function in the check-node processing?\nOptions:\noption 1: Normalized min-sum\noption 2: Off-set min-sum\noption 3: Belief propagation\noption 4: Successive cancellation\noption 5: Sequential decoding\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAnswer: option 3: Belief propagation\nExplanation: The belief propagation decoding algorithm of low-density parity-check codes involves a complex non-linear function in the check-node processing.<|eot_id|>'

# Training

In [29]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.10.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [30]:
# del model

In [13]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 16,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 150,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

/home/arimatea/.local/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:401: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  super().__init__(
max_steps is given, it will override any value given in num_train_epochs


In [14]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [15]:
tokenizer.decode(trainer.train_dataset[0]["input_ids"])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the purpose of sequence and cyclic shift hopping in PUCCH formats 0, 1, 3, and 4? [3GPP Release 17]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSequence and cyclic shift hopping are used in PUCCH formats 0, 1, 3, and 4 to reduce the interference between PUCCH transmissions. These hopping mechanisms ensure that different UEs use different sequences and cyclic shifts, increasing the orthogonality of the transmitted signals and reducing inter-UE interference.<|eot_id|>'

In [16]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[0]["labels"]])

'                                                                      \n\nSequence and cyclic shift hopping are used in PUCCH formats 0, 1, 3, and 4 to reduce the interference between PUCCH transmissions. These hopping mechanisms ensure that different UEs use different sequences and cyclic shifts, increasing the orthogonality of the transmitted signals and reducing inter-UE interference.<|eot_id|>'

## Show current memory stats


In [35]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 3050 Ti Laptop GPU. Max memory = 3.712 GB.
2.768 GB of memory reserved.


## Initial Loss

In [36]:
trainer.train_dataset[0].items()

dict_items([('input_ids', [128000, 128006, 9125, 128007, 271, 38766, 1303, 33025, 2696, 25, 6790, 220, 2366, 18, 198, 15724, 2696, 25, 220, 1627, 5887, 220, 2366, 19, 271, 128009, 128006, 882, 128007, 271, 3923, 374, 279, 7580, 315, 8668, 323, 77102, 6541, 93338, 304, 393, 5576, 2198, 20447, 220, 15, 11, 220, 16, 11, 220, 18, 11, 323, 220, 19, 30, 510, 18, 38, 4505, 17836, 220, 1114, 60, 128009, 128006, 78191, 128007, 271, 14405, 323, 77102, 6541, 93338, 527, 1511, 304, 393, 5576, 2198, 20447, 220, 15, 11, 220, 16, 11, 220, 18, 11, 323, 220, 19, 311, 8108, 279, 32317, 1990, 393, 5576, 2198, 92096, 13, 4314, 93338, 24717, 6106, 430, 2204, 549, 17812, 1005, 2204, 24630, 323, 77102, 29735, 11, 7859, 279, 30299, 540, 263, 2786, 315, 279, 34699, 17738, 323, 18189, 958, 12, 2279, 32317, 13, 128009]), ('attention_mask', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [32]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Configura o collator e DataLoader
collator = DataCollatorForSeq2Seq(tokenizer)
loader = DataLoader(trainer.train_dataset, 
                    batch_size=2,  # Tamanho do batch escolhido
                    collate_fn=collator, 
                    num_workers=2)

# Variáveis para armazenar a loss total e o número de exemplos
total_loss = 0.0
num_batches = 0

# Coloca o modelo em modo de avaliação
model.eval()

# Desativa o cálculo de gradiente para economizar memória
with torch.no_grad():
    for batch in tqdm(loader, desc="Calculating initial loss"):
        # Move o batch para a GPU (se disponível)
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)
        
        # Acumula a loss
        total_loss += outputs.loss.item()
        num_batches += 1

# Calcula a loss média
average_loss = total_loss / num_batches
print(f"Initial mean loss: {average_loss}")


Calculating initial loss: 100%|██████████| 1000/1000 [10:09<00:00,  1.64it/s]

Initial mean loss: 1.7596246767640114


## Training Session

In [37]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 16
\        /    Total batch size = 32 | Total steps = 150
 "-____-"     Number of trainable parameters = 24,313,856


  0%|          | 0/150 [00:00<?, ?it/s]

{'loss': 1.7554, 'grad_norm': 1.1779676675796509, 'learning_rate': 4e-05, 'epoch': 0.02}
{'loss': 1.795, 'grad_norm': 1.1981481313705444, 'learning_rate': 8e-05, 'epoch': 0.03}
{'loss': 1.6159, 'grad_norm': 1.178970456123352, 'learning_rate': 0.00012, 'epoch': 0.05}
{'loss': 1.5085, 'grad_norm': 1.2041479349136353, 'learning_rate': 0.00016, 'epoch': 0.06}
{'loss': 1.4737, 'grad_norm': 1.4692350625991821, 'learning_rate': 0.0002, 'epoch': 0.08}
{'loss': 1.0093, 'grad_norm': 0.9795082211494446, 'learning_rate': 0.00019862068965517243, 'epoch': 0.1}
{'loss': 1.23, 'grad_norm': 0.4913758933544159, 'learning_rate': 0.00019724137931034484, 'epoch': 0.11}
{'loss': 1.1138, 'grad_norm': 0.37414249777793884, 'learning_rate': 0.00019586206896551723, 'epoch': 0.13}
{'loss': 1.0801, 'grad_norm': 0.32851642370224, 'learning_rate': 0.00019448275862068965, 'epoch': 0.14}
{'loss': 1.1364, 'grad_norm': 0.5934054851531982, 'learning_rate': 0.0001931034482758621, 'epoch': 0.16}
{'loss': 0.7544, 'grad_norm

## Show final memory and time stats

In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [39]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

## Loss Pos training

In [19]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

# Configure the collator and DataLoader
collator = DataCollatorForSeq2Seq(tokenizer)
loader = DataLoader(
    trainer.train_dataset, 
    batch_size=2,  # Chosen batch size
    collate_fn=collator, 
    num_workers=2
)

# Variables to store the total loss and the number of valid batches
total_loss = 0.0
num_batches = 0

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation to save memory
with torch.no_grad():
    for batch in tqdm(loader, desc="Calculating loss post-training"):
        # Move the batch to GPU (if available)
        batch = {k: v.to(model.device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Check if the loss is NaN
        if not torch.isnan(outputs.loss):
            total_loss += outputs.loss.item()
            num_batches += 1
            # print(outputs.loss.item())

print(f"Number of valid batches: {num_batches}")

# Calculate the average loss
if num_batches > 0:
    average_loss = total_loss / num_batches
    print(f"Post-training mean loss: {average_loss}")
else:
    print("No valid batches found.")


Calculating loss post-training: 100%|██████████| 1000/1000 [06:08<00:00,  2.71it/s]

Number of valid batches: 1000
Post-training mean loss: 0.7625631860084832


# Inference

## Question with option

In [71]:
dataset[5]['conversations'][0]['value']

'Question: Which decoding algorithm for low-density parity-check codes involves a complex non-linear function in the check-node processing?\nOptions:\noption 1: Normalized min-sum\noption 2: Off-set min-sum\noption 3: Belief propagation\noption 4: Successive cancellation\noption 5: Sequential decoding\n'

In [72]:
question = dataset[5]['conversations'][0]['value']

In [73]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Which decoding algorithm for low-density parity-check codes involves a complex non-linear function in the check-node processing?\nOptions:\noption 1: Normalized min-sum\noption 2: Off-set min-sum\noption 3: Belief propagation\noption 4: Successive cancellation\noption 5: Sequential decoding\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAnswer: option 3: Belief propagation\nExplanation: Belief propagation decoding involves complex non-linear functions in the check-node processing.<|eot_id|>']

## Question with no option

In [74]:
dataset[0]['conversations'][0]['value']

'What is the purpose of sequence and cyclic shift hopping in PUCCH formats 0, 1, 3, and 4? [3GPP Release 17]'

In [77]:
question = dataset[0]['conversations'][0]['value']

In [78]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nWhat is the purpose of sequence and cyclic shift hopping in PUCCH formats 0, 1, 3, and 4? [3GPP Release 17]<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nSequence and cyclic shift hopping is used in PUCCH formats 0, 1, 3, and 4 to transmit data sequences over frequency domain.<|eot_id|>']

# Save model

In [49]:
model.save_pretrained("../Models/llama_3.2_FT_lora_2000_questions", safe_serialization=False)
tokenizer.save_pretrained("../Models/llama_3.2_FT_lora_2000_questions")

('../Models/llama_3.2_FT_lora_2000_questions/tokenizer_config.json',
 '../Models/llama_3.2_FT_lora_2000_questions/special_tokens_map.json',
 '../Models/llama_3.2_FT_lora_2000_questions/tokenizer.json')

In [45]:
# model.save_pretrained_merged("model_3.2_lora_4bits", tokenizer, save_method = "merged_16bit",)


# Load Model

In [4]:
max_seq_length = 8192 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [5]:
model_path = "../Models/llama_3.2_FT_lora_2000_questions"
# model_path = "model_3.2_lora_4bits"

# Carregar o modelo e o tokenizador separadamente
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_path,
    max_seq_length=max_seq_length,
    dtype = dtype,
    load_in_4bit=load_in_4bit
)

==((====))==  Unsloth 2024.10.6: Fast Llama patching. Transformers = 4.46.0.
   \\   /|    GPU: NVIDIA GeForce RTX 3050 Ti Laptop GPU. Max memory: 3.712 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.6. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth 2024.10.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [5]:
model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 3072)
        (layers): ModuleList(
          (0-27): 28 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=3072, out_features=3072, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=3072, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=3072, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear4bit(
      

# Test Loaded Model

In [8]:
from unsloth.chat_templates import get_chat_template

question = dataset[5]['conversations'][0]['value']

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    # {"role": "user", "content": "How much is 1+1?"},
    {"role": "user", "content": question},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuestion: Which decoding algorithm for low-density parity-check codes involves a complex non-linear function in the check-node processing?\nOptions:\noption 1: Normalized min-sum\noption 2: Off-set min-sum\noption 3: Belief propagation\noption 4: Successive cancellation\noption 5: Sequential decoding\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nAnswer: option 3: Belief propagation\nExplanation: The belief propagation algorithm for low-density parity-check codes involves a complex non-linear function in the check-node processing, specifically the Q-function.<|eot_id|>']